# PS 3
## Question 2: Bi-Linear Models and SGD

**Your Name:** [Insert Your Name here]

**People I discussed this question with:** [Put "None" if you didn't discuss this with anyone, otherwise list the names of folks you talked about this problem set with]

# Bilinear Models for Recommendation

For this problem, you will derive a very simple recommendation system that uses a combination of unsupervised and supervised approachs and demonstrates use of Stochastic Gradient Descent.

Specifically, in class we discussed recommender models of the form:
$$
f(user,movie) = \langle v_u,v_m \rangle + b_u + b_m + \mu
$$

where $v$ is a vector that represents a user's or movie's location in an N-Dimensional space, $b$ is a vector that represents a specific "bias" term fo
r each movie and user, and $\mu$ is a scalar that represents a kind of global anchor or base score (i.e., a sort of average movie rating). This means that each user has two vectors (e.g., $v_{\mathrm{jack~smith}}$ and $b_{\mathrm{jack~smith}}$), and each movie has two vectors (e.g., $v_{\mathrm{Avengers}}$ and $b_{\mathrm{Avengers}}$), with each of those vectors being N-Dimensional (in class we used two dimensions). For this, we constructed a loss function as follows:
$$
Cost = Loss + Penalty
$$
where
$$
Loss = \Sigma_{(u,m)\in \mathrm{Ratings}} \frac{1}{2}\left( \langle v_u,v_m \rangle + b_u + b_m + \mu - y_{u,m}\right)^2
$$
and
$$
Penalty = \frac{\lambda}{2}\left(\Sigma_u \left[\| v_u\|^2_2 + b_u^2\right] + \Sigma_m \left[\|v_m\|^2_2 + b_m^2\right]\right)
$$


# Task 1: Analytical Gradients
To use stochastic gradient descent, we first need to write down the gradients. Using the above cost function (including both the loss and penalty), compute the following partial derivatives:

$$
\frac{d}{dv_u}<v_u,v_m> = \frac{d}{dv_u}v_u^Tv_m = 1*v_m + v_u^T*0
$$
$$
\frac{d}{dx} ||x||^2_2 = 2||x||_2
$$
Assuming my calculus is correct

$$
\frac{\partial \textrm{Cost}}{\partial v_u } = \Sigma_{(u,m)\in Ratings} v_m(b_m + b_u + \mu - y_{u,m} + v_u^Tv_m) + \lambda\Sigma_u ||v_u||_2
$$

$$
\frac{\partial \textrm{Cost}}{\partial v_m } = \Sigma_{(u,m)\in Ratings} v_u(b_m + b_u + \mu - y_{u,m} + v_u^Tv_m) + \lambda\Sigma_m ||v_m||_2
$$

$$
\frac{\partial \textrm{Cost}}{\partial b_u} = \Sigma_{(u,m)\in Ratings} (b_m + b_u + \mu - y_{u,m} + v_u^Tv_m) + \lambda\Sigma_u b_u
$$

$$
\frac{\partial \textrm{Cost}}{\partial b_m} = \Sigma_{(u,m)\in Ratings} (b_m + b_u + \mu - y_{u,m} + v_u^Tv_m) + \lambda\Sigma_m b_m
$$

$$
\frac{\partial \textrm{Cost}}{\partial \mu} = \Sigma_{(u,m)\in Ratings} (b_m + b_u + \mu + y_{u,m} + v_u^Tv_m)
$$

You can either do this directly in the notebook using LaTeX notation, or via a scanned image. Note: Recall that the partial derivative of e.g. Nicholas's rating on Titanic with respect to the user Mark would be zero. When computing your SGD updates, consider how this might impact individual terms for users and movies in the loss function.

# Task 2: Stochastic Gradient Descent
Now you are actually going to implement SGD on this type of model and optimize it until convergence on a toy dataset. To simplify the implementation, we'll actually make the model a little simpler than the one you derived updates for in task 1. Specifically, we'll just use:

$$
Cost = \Sigma_{(u,m)\in \mathrm{Ratings}} \frac{1}{2}\left( \langle v_u,v_m \rangle + \mu - y_{u,m}\right)^2 + \frac{\lambda}{2}\left(\| v_u\|^2_2 + \|v_m\|^2_2\right)
$$

$$
\begin{bmatrix} v_u \\ v_m \end{bmatrix} = \begin{bmatrix} v_u \\ v_m \end{bmatrix} - \alpha\begin{bmatrix} \Sigma_{(u,m)\in Ratings}v_m(\mu - y_{u,m} + v_u^Tv_m) + \lambda ||v_u||_2 \\ \Sigma_{(u,m)\in Ratings}v_u(\mu - y_{u,m} + v_u^Tv_m) + \lambda ||v_m||_2 \end{bmatrix} = v - \alpha \nabla Cost(v)
$$

This way all you have to estimate is two vectors — $v_u$ for each user and $v_m$ for each movie — and $\mu$ — a scalar value similar to an average rating. For simplicity, we'll assume here that the size of the latent space (K) is 2 (i.e., the length of each  $v_u$ & $v_m$).

Using your above gradients, write down the update equations for each vector using stochastic gradient descent. Once you have done this, implement those update equations in code like we did in the in-class notebook. For simplicity, you can just use a constant step size $\alpha$ if you wish, though you may change this if you want. Note: depending on exactly how you implement your model and what batch size you use, i.e., one point at a time, or some subset of data points, values of $\alpha$ anywhere between around 0.7 and 0.01 should be sufficient to converge the model in under 1000 epochs, i.e., passes through the dataset. If you implement more advanced tricks covered in some optional readings this can converge much faster, but that is not necessary for this assignment, and it does not matter to me how quickly your model coverges, so long as it does so.

Use the below small sample dataset of movie ratings for five users and six movies to perform stochastic gradient descent to update those vectors until your model converges. To initialize your SGD, you can use the initial weights/terms we provide below, or you can initialize the model any other way you wish -- the exact initialization should not make a big difference here.

In [1]:
# Your Code below!

In [2]:
import numpy as np
import pandas as pd
missing_ratings = pd.read_csv('missing.csv')
ratings = pd.read_csv('ratings.csv')
ratings

,movie,user,ratings
0,The Avengers,Alex,3.0
1,The Avengers,Priya,3.5
2,The Avengers,Yichen,3.5
3,When Harry Met Sally,Alex,3.0
4,When Harry Met Sally,Sally,4.5
5,When Harry Met Sally,Priya,3.0
6,When Harry Met Sally,Yichen,3.0
7,Silence of the Lambs,Alex,3.0
8,Silence of the Lambs,Sally,4.0
9,Silence of the Lambs,Juan,3.5


In [3]:
# Alternatively, if you prefer, you can convert it into numpy first:
ratings_numpy = ratings.to_numpy()
ratings_numpy

array([['The Avengers', 'Alex', 3.0],
       ['The Avengers', 'Priya', 3.5],
       ['The Avengers', 'Yichen', 3.5],
       ['When Harry Met Sally', 'Alex', 3.0],
       ['When Harry Met Sally', 'Sally', 4.5],
       ['When Harry Met Sally', 'Priya', 3.0],
       ['When Harry Met Sally', 'Yichen', 3.0],
       ['Silence of the Lambs', 'Alex', 3.0],
       ['Silence of the Lambs', 'Sally', 4.0],
       ['Silence of the Lambs', 'Juan', 3.5],
       ['Silence of the Lambs', 'Priya', 3.0],
       ['Silence of the Lambs', 'Yichen', 2.5],
       ['Shawshank Redemption', 'Juan', 2.5],
       ['Shawshank Redemption', 'Priya', 4.0],
       ['Shawshank Redemption', 'Yichen', 4.0],
       ['The Hangover', 'Alex', 3.0],
       ['The Hangover', 'Sally', 3.5],
       ['The Hangover', 'Priya', 3.0],
       ['The Hangover', 'Yichen', 2.5],
       ['The Godfather', 'Alex', 3.0],
       ['The Godfather', 'Priya', 3.5]], dtype=object)

Let's initialize the vectors to some random numbers, and $\mu$ to 2.5

In [4]:
K=2
user_names = ratings['user'].unique()
movie_names = ratings['movie'].unique()
mu= 2.5
# Setting the seed of the random generator to a value so that everyone sees the same initialization
# should should be able to comment out the below with no ill-effects on whatever model you implement
# this may just help us in office hours if folks have difficulty implementing things
np.random.seed(0)
V = pd.DataFrame(np.random.random((len(user_names)+len(movie_names),K)),index=np.hstack([user_names,movie_names]))
print(V)

                             0         1
Alex                  0.548814  0.715189
Priya                 0.602763  0.544883
Yichen                0.423655  0.645894
Sally                 0.437587  0.891773
Juan                  0.963663  0.383442
The Avengers          0.791725  0.528895
When Harry Met Sally  0.568045  0.925597
Silence of the Lambs  0.071036  0.087129
Shawshank Redemption  0.020218  0.832620
The Hangover          0.778157  0.870012
The Godfather         0.978618  0.799159


In [5]:
# Here is one example of how to go through rows of a ratings matrix
for index, rating in ratings.iterrows():
    user  = rating['user']
    movie = rating['movie']
    score = rating['ratings']
    print(f"{user} gave {movie} a score of {score}")         

Alex gave The Avengers a score of 3.0
Priya gave The Avengers a score of 3.5
Yichen gave The Avengers a score of 3.5
Alex gave When Harry Met Sally a score of 3.0
Sally gave When Harry Met Sally a score of 4.5
Priya gave When Harry Met Sally a score of 3.0
Yichen gave When Harry Met Sally a score of 3.0
Alex gave Silence of the Lambs a score of 3.0
Sally gave Silence of the Lambs a score of 4.0
Juan gave Silence of the Lambs a score of 3.5
Priya gave Silence of the Lambs a score of 3.0
Yichen gave Silence of the Lambs a score of 2.5
Juan gave Shawshank Redemption a score of 2.5
Priya gave Shawshank Redemption a score of 4.0
Yichen gave Shawshank Redemption a score of 4.0
Alex gave The Hangover a score of 3.0
Sally gave The Hangover a score of 3.5
Priya gave The Hangover a score of 3.0
Yichen gave The Hangover a score of 2.5
Alex gave The Godfather a score of 3.0
Priya gave The Godfather a score of 3.5


In [6]:
# Here is an example of one way to access rows of V
for index, rating in ratings.iterrows():
    user  = rating['user']
    movie = rating['movie']
    print(f"{user}'s location in V is {V.loc[user].to_numpy()}.")
    print(f"{movie}'s location in V is {V.loc[movie].to_numpy()}.")
    print()
    
# You could also do it in Numpy directly, which will likely lead to much faster SGD updates,
# but that shouldn't be necessary for problems of this size. Up to you!

Alex's location in V is [0.5488135  0.71518937].
The Avengers's location in V is [0.79172504 0.52889492].

Priya's location in V is [0.60276338 0.54488318].
The Avengers's location in V is [0.79172504 0.52889492].

Yichen's location in V is [0.4236548  0.64589411].
The Avengers's location in V is [0.79172504 0.52889492].

Alex's location in V is [0.5488135  0.71518937].
When Harry Met Sally's location in V is [0.56804456 0.92559664].

Sally's location in V is [0.43758721 0.891773  ].
When Harry Met Sally's location in V is [0.56804456 0.92559664].

Priya's location in V is [0.60276338 0.54488318].
When Harry Met Sally's location in V is [0.56804456 0.92559664].

Yichen's location in V is [0.4236548  0.64589411].
When Harry Met Sally's location in V is [0.56804456 0.92559664].

Alex's location in V is [0.5488135  0.71518937].
Silence of the Lambs's location in V is [0.07103606 0.0871293 ].

Sally's location in V is [0.43758721 0.891773  ].
Silence of the Lambs's location in V is [0.0710

# Train your Bilinear Model using SGD

In [12]:
# Your Model building and training code here!

# Assessing your accuracy
Let's predict the ratings for the missing entries using our (randomly initialized) model.

In [9]:
for index, rating in missing_ratings.iterrows():
    user  = rating['user']
    movie = rating['movie']
    prediction = np.dot(V.loc[user],V.loc[movie])+mu
    print(f"Prediction: {user} will rate {movie}: {prediction:.2f}")

Prediction: Sally will rate The Avengers: 3.32
Prediction: Juan will rate The Avengers: 3.47
Prediction: Juan will rate When Harry Met Sally: 3.40
Prediction: Alex will rate Shawshank Redemption: 3.11
Prediction: Sally will rate Shawshank Redemption: 3.25
Prediction: Juan will rate The Hangover: 3.58
Prediction: Sally will rate The Godfather: 3.64
Prediction: Juan will rate The Godfather: 3.75
Prediction: Yichen will rate The Godfather: 3.43
